# **변형(TRANSFORM)**


---


데이터가 머신러닝 알고리즘 학습에 항상 최적화된 상태는 아니기 때문에, 형태를 변형(transform)해서 학습에 적합한 형태로 만들어야 할 때가 있음.   
<br>
모든 TorchVision 데이터 셋들은 **특징을 변경하기 위한 매개변수인 *transform* **, **정답을 변경하기 위한 *target_transform* **을 가지고 있음. *torchvision.transforms* 모듈은 주로 사용하는 몇 가지 transform을 제공함.
<br>

예를 들면, FashionMNIST의 feature는 PIL Image 형식이고, label은 정수임. 학습을 하려면 이를 정규화(normalize)된 텐서 형태의 feature와 one-hot으로 부호화(encode)된 텐서 형태의 label로 바꿔야 함. 이렇게 변환하기 위해서, **ToTensor**와 **Lambda**를 사용함.

## **ToTensor()**

PIL Image나, NumPy ndarray를 **FloatTensor**로 변환하고, 이미지 픽셀의 크기(intensity)값을 **[0.,1.]범위로 비례하여 조정**.

## **Lambda 변형(Transform)**

사용자 정의 람다(lambda)함수 적용.   
여기서는 정수를 원-핫 인코딩된 텐서로 바꾸는 함수를 정의함.

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="./",
    train=True,
    download=True,
    transform = ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))  #먼저 10개의 0으로 이루어진 zero tensor(float형)를 만들고,
                                                                                                                   #scatter_를 호출해 주어진 정답 y에 해당하는 인덱스에 value=1을 할당.
)

100%|██████████| 26421880/26421880 [00:01<00:00, 13556852.24it/s]


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 268544.27it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4941445.11it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23317793.73it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



### **여기서 잠깐!**

### **One-hot vector와 scatter**
공부 자료: https://wikidocs.net/59678
           https://scored-a-belter.tistory.com/10
    

#### **⚡ One-hot vector**
One-hot vector는 해당 선택지에만 1을 가지고, 나머지는 0으로 표현된 벡터를 말함.

예)  고양이[1,0]   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;강아지[0,1]
<br><br>
**❓ 다중 클래스 분류 문제에서 원-핫 벡터를 사용하는 이유?**   

예를 들어, Tomato, Banana, Apple을 각각 1,2,3으로 라벨링 했다고 할 때, 평균 제곱 오차(MSE)를 구할 때 문제가 일어날 수 있다.
<img src="https://user-images.githubusercontent.com/10937193/58108494-e7bc7580-7c26-11e9-90a1-b988522a0b64.png">
직관적 비교를 위해 제곱 오차만 보고 따져 보면,
<br><br>
>실제값: Tomato   
예측값: Banana   
제곱 오차: (2-1)²= 1   

>실제값: Tomato   
예측값: Apple   
제곱 오차: (3-1)²= 4

<br>

즉, **Tomato가 Apple보다 Banana랑 유사도가 높다**는 결과가 나와버림.

물론, [10대, 20대, 30대], [첫 번째, 두 번째, 세 번째]처럼 feature가 아예 순서의 의미도 같이 가지고 있는 경우는 괜찮겠지만, Tomato, Apple, Banana는 **순서를 가지지 않으므로, 각 클래스간 오차가 동일한게 맞음**. 이 feature들을 임의로 정수로 라벨링 하는 것은 좋지 않음.
<br><br>

그럼 어떻게 해야 할까?   
**원-핫 인코딩**으로 해당 문제를 풀면, 각 label간의 오차를 동일하게 맞출 수 있음.

예) ((1,0,0) - (0,1,0))²= (1-0)²+ (0-1)²+ (0-0)²=2    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ((1,0,0) - (0,0,1))²= (1-0)²+ (0-0)²+ (0-1)²=2

다르게 말하자면, 모든 클래스에 대해 원-핫 인코딩을 통해 얻은 원-핫 벡터들은, **모든 쌍에 대해서 유클리드 거리를 구해도 전부 유클리드 거리가 동일**함. 이처럼 원-핫 벡터로 label 표현은 **무작위성**을 가짐. 때문에 때로는 단어의 유사성을 구할수 없다는 단점이 있음.
<br><br>

#### **⚡ .scatter()**

정해진 기준축을 따라 index자리에 값을 흩뿌리는 메서드.   
***torch.scatter(input, dim, index, src)*** -> Tensor



```
src = torch.arange(1, 11).reshape((2, 5))
src
>> tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])
        
        
index = torch.tensor([[0, 1, 2, 0]])
torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src)
>> tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])
```

*torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src)*
1. size가 [3,5]인 zero tensor를 만든다
2. 문자형 타입은 src의 타입으로 덮어 쓴다
3. zero tensor에 src의 값을 이제 흩뿌릴건데,
4. dim = 0이므로 기준은 '어떤 행으로 보낼지'가 된다
    (dim =1은 열이 기준)
5. index에 따라 src 값이 흩뿌려진다.
6. [0, 1, 2, 0], dim=0 이므로,
   src의 첫 번째 값 1은 0행 0열로,   
   src의 두 번째 값 2은 1행 1열로,   
   src의 세 번째 값 3은 2행 2열로,   
   src의 첫 번째 값 0은 0행 3열로 흩어진다.
7. [[1, 0, 0, 4, 0],   
    [0, 2, 0, 0, 0],   
    [0, 0, 3, 0, 0]]



